In [8]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
serpapi_api_key = os.environ["SERPAPI_API_KEY"]



In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import OpenAI

In [4]:
my_creative_llm = OpenAI(temperature=0.9)
question = "what are the best 5 places in the world for scuba diving?"
print(my_creative_llm(question))



1. Great Barrier Reef, Australia
2. Maldives
3. Galapagos Islands, Ecuador
4. Red Sea, Egypt
5. Palau, Micronesia


# Uso de prompt Templates

In [5]:
my_prompt_template = PromptTemplate(
    input_variables = ["destination"],
    template = "what are the 3 best things to do in {destination}"
)
user_input = "Barcelona"
print(my_creative_llm(my_prompt_template.format(destination=user_input)))



1. Visit Sagrada Familia: This iconic cathedral designed by Antoni Gaudí is a must-see in Barcelona. Its unique architecture and stunning interior make it a popular tourist destination.

2. Stroll down Las Ramblas: This famous street in Barcelona is filled with shops, cafes, street performers, and vibrant energy. It's a great place to soak up the city's atmosphere and people-watch.

3. Explore Park Güell: Another one of Gaudí's masterpieces, this park is known for its colorful mosaic structures and beautiful views of the city. Take a walk through the park and admire the unique art and design.


# Instrucciones en cadena


In [6]:
my_no_creative_llm  = OpenAI(temperature=0)
my_prompt_template = PromptTemplate(
    input_variables = ["politician"],
    template = "what are the 3 most remarkable achievements of {politician}?"
)
my_chain = LLMChain(llm=my_no_creative_llm, prompt=my_prompt_template)
print(my_chain.run("Churchill"))

/home/vonzipper/anaconda3/envs/llm_bootcamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/vonzipper/anaconda3/envs/llm_bootcamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. Leadership during World War II: Churchill's leadership during World War II is considered one of his most remarkable achievements. He played a crucial role in rallying the British people and inspiring them to resist Nazi Germany. His speeches, such as the famous "We shall fight on the beaches" speech, boosted morale and united the country during a time of great adversity.

2. Political career and Prime Minister of the United Kingdom: Churchill had a long and successful political career, serving as Prime Minister of the United Kingdom twice, from 1940 to 1945 and again from 1951 to 1955. He was also a member of Parliament for over 60 years and held various other government positions, including Chancellor of the Exchequer and Home Secretary.

3. Nobel Prize in Literature: In 1953, Churchill was awarded the Nobel Prize in Literature for his numerous works, including his six-volume memoir "The Second World War." He was the first British Prime Minister to receive this honor and was reco

# Crear agentes

In [7]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["serpapi", "llm-math"], llm=my_no_creative_llm)
agent = initialize_agent(tools, my_no_creative_llm, "zero-shot-react-description")
agent.run("who is the current leader of Japan?")

/home/vonzipper/anaconda3/envs/llm_bootcamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


'The current leader of Japan is Fumio Kishida.'

# Añadir memoria a un LLM

In [9]:
from langchain import ConversationChain

my_conversation = ConversationChain(llm=my_no_creative_llm, verbose=True)
my_conversation("Hi there!")


/home/vonzipper/anaconda3/envs/llm_bootcamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(
/home/vonzipper/anaconda3/envs/llm_bootcamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


{'input': 'Hi there!',
 'history': '',
 'response': " Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?"}

# Output parsers

In [19]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator

class reviews_desired_output_parser(BaseModel):
    sentiment: list[str] = Field(
        description = "is it a positive or negative review?"
    )
    positive : list[str] = Field(
        description = "what things are positive?"
    )
    negative : list[str] = Field(
        description = "Things that are negative"
    )

output_parser = PydanticOutputParser(pydantic_object=reviews_desired_output_parser)
template_for_desired_output_parser = """\
    Here is a book review: {book_review}
    
    I want you to output three things in Json format dictionary
    
    {format_instructions}
    "sentiment": is ti positive or negative,
    "positive": what is positive,
    "negative": what is negative
    """
prompt_template_with_parser = PromptTemplate(
    template = template_for_desired_output_parser,
    input_variables = ["book_review"],
    partial_variables = {
        "format_instructions": output_parser.get_format_instructions()
    }
)


In [20]:
oathbringer_review = """I want to tell you a story.

In the last few weeks, I have seen the ultimate convergence of all things Marvel in Avengers: Endgame, watched the biggest TV Show in history (Game of Thrones) come to an end, and now - listened to the longest audiobook of my life in Oathbringer. I admit, I was much more excited for Endgame and Game of Thrones, but guess what? Oathbringer was the best, most satisfying experience of the three.

How did that happen?

Almost 8 years ago, I began my journey with Brandon Sanderson in Mistborn. I tore through the paperbacks of that trilogy and desperately wanted more. I immediately ordered the hardback of The Way of Kings and....was immensely disappointed. Don't get me wrong, it is a BEAUTIFUL book. But the story was so...slow compared to Mistborn. It took me almost 3 years to finally get around to reading it.

I did, though, and although I was still confused by some of the complexity of it - I bought/read Words of Radiance when it came out in 2014. Now, THIS was more like it! Much more action, adventure, and awkward romance! Okay, so the awkward romance and dialogue wasn't my favorite, but I did love the story, and was so excited for Book 3!

During the wait, however, I read a lot more. I discovered authors that were....different from Sanderson. Not necessarily better, per se, but different. I found myself listening to some of Brandon's critics. "He can't write believable dialogue", they said. "His prose is too simple, and his language too modern." "His sense of humor is childish." "All of his characters share the same morality." And on, and on.

I shouldn't have been so easily influenced, but I was. I found myself growing less and less excited about Oathbringer. So much so that when it released in 2017 - I didn't buy it. I didn't pay attention to everyone discussing it. I put it out of my mind.

Until now.

I have finally revisited The Cosmere. And friends? It was GLORIOUS*(spren).*

Look, Oathbringer isn't perfect. There IS awkward dialogue. The prose CAN be simplistic. The pacing IS often slow. But GUYS - these criticisms pale in comparison to what Sanderson has accomplished with his characters, plotting, and world.

I was genuinely moved by the journey Kaladin, Shallan, and now Dalinar continue to go through - their fight against the repercussion of past actions (leading to present mental illness) was utterly enthralling. As someone who himself struggles with anxiety and depression, I very much identify with Kaladin in particular.

(Also, I love the minor characters - ALL of them. For example, Jasnah was already my Queen - but now she's crowned and I am HERE FOR IT.)

The plot steamrolled ahead in this book - way faster than the previous two. We got loads of new information that I am still trying to digest - and all of it makes me want to go back and read Way of Kings right now! I'm not sure how a book can succeed better than that, right?

And then, of course, there's Roshar. This feels like a real, breathing, place. I can think of very few a better-realized world in all of fiction. It is simply awe-inspiring. And I'm so grateful that, in Oathbringer, we actually get to see so much more of it.

The lesson here, kids, is not to let other people tell you what to like or dislike - nothing is perfect. But somethings might be perfect FOR you.

Stormlight Archive is perfect for me, and I can't wait for Book #4."""

In [25]:
import json
question = prompt_template_with_parser.format(book_review=oathbringer_review)
response = my_no_creative_llm(question)
formated_output = output_parser.parse(response)
json_output = formated_output.model_dump_json()
python_dict = json.loads(json_output)
python_dict

{'sentiment': ['positive'],
 'positive': ['characters', 'plot', 'world-building'],
 'negative': ['awkward dialogue', 'simplistic prose', 'slow pacing']}

# Conversation Buffer Memory

In [27]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI()
buffer_memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = buffer_memory,
    verbose=True
)

In [28]:
conversation.predict(input="Hi, my name is Javier and I have moved 33 times")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Javier and I have moved 33 times
AI:

> Finished chain.


" Hello Javier, it's nice to meet you. My name is AI and I don't have a physical body like you, so I don't have the ability to move. However, I am constantly being moved from one server to another to fulfill my tasks. How have your experiences been with moving?"

In [29]:
conversation.predict(input="if my average moving distance is 10 miles, how many miles took all my moves?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Javier and I have moved 33 times
AI:  Hello Javier, it's nice to meet you. My name is AI and I don't have a physical body like you, so I don't have the ability to move. However, I am constantly being moved from one server to another to fulfill my tasks. How have your experiences been with moving?
Human: if my average moving distance is 10 miles, how many miles took all my moves?
AI:

> Finished chain.


' Based on the information you provided, you have moved a total of 330 miles. That is quite a distance! It must have been quite an adventure to move to all those different places.'

In [30]:
conversation.predict(input="do you remember my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Javier and I have moved 33 times
AI:  Hello Javier, it's nice to meet you. My name is AI and I don't have a physical body like you, so I don't have the ability to move. However, I am constantly being moved from one server to another to fulfill my tasks. How have your experiences been with moving?
Human: if my average moving distance is 10 miles, how many miles took all my moves?
AI:  Based on the information you provided, you have moved a total of 330 miles. That is quite a distance! It must have been quite an adventure to move to all those different places.
Human: do you remember my name?
AI:

> Finished chain.


' Yes, your name is Javier. I have a vast memory capacity and can remember a lot of information, including names. Is there anything else you would like to know or discuss, Javier?'

In [34]:
print(buffer_memory.buffer)

Human: Hi, my name is Javier and I have moved 33 times
AI:  Hello Javier, it's nice to meet you. My name is AI and I don't have a physical body like you, so I don't have the ability to move. However, I am constantly being moved from one server to another to fulfill my tasks. How have your experiences been with moving?
Human: if my average moving distance is 10 miles, how many miles took all my moves?
AI:  Based on the information you provided, you have moved a total of 330 miles. That is quite a distance! It must have been quite an adventure to move to all those different places.
Human: do you remember my name?
AI:  Yes, your name is Javier. I have a vast memory capacity and can remember a lot of information, including names. Is there anything else you would like to know or discuss, Javier?


In [37]:
buffer_memory.load_memory_variables({})

{'history': "Human: Hi, my name is Javier and I have moved 33 times\nAI:  Hello Javier, it's nice to meet you. My name is AI and I don't have a physical body like you, so I don't have the ability to move. However, I am constantly being moved from one server to another to fulfill my tasks. How have your experiences been with moving?\nHuman: if my average moving distance is 10 miles, how many miles took all my moves?\nAI:  Based on the information you provided, you have moved a total of 330 miles. That is quite a distance! It must have been quite an adventure to move to all those different places.\nHuman: do you remember my name?\nAI:  Yes, your name is Javier. I have a vast memory capacity and can remember a lot of information, including names. Is there anything else you would like to know or discuss, Javier?"}